# Mesures de désordre lors de mélanges d'un paquet de cartes: Implementation des melanges et du calcul d'entropies

## Table des matières

- Introduction
    - Entropie à 1 position: $S_1$
    - Entropie à n positions: $S_n$ 
- Implémentation des mélanges
    - Mélange "haut-bas"
    - Mélange "alterné"
- Implémentation des entropies positionnelles
    - Entropie à 1 position
    - Valeurs typiques de l'entropie à 1 position pour différentes  distributions finales
    - Entropie à 2 positions
    - Valeurs typiques de l'entropie à 2 positions pour différentes  distributions finales
- Sources d'erreur dans les mesures d'entropie
    - Nombre insuffisant de simulations
- Stockage dans des fichiers 

## Introduction

Dans le but d'en évaluer la qualité, nous tenons à mesurer le désordre produit par un mélange de cartes.
Pour ce faire, nous introduisons des fonctions "entropies positionneles" associées au mélange considéré.

### Entropie à 1 position: $S_1$

La fonction entropie à 1 position, notée $S_1(p)$, mesure le désordre engendré par le mélange sur un paquet avec une position $p$ connue. Le paquet initial est ainsi constitué d'une seule carte connue, à la position $p$. Le paquet est ensuite mélangé et une nouvelle configuration est ainsi obtenue. Les nouvelles configurations sont caractérisées par la carte connue à une certaine position, disons $q$, et les autres cartes toujours inconnues. Une entropie peut être associée à ce mélange en considérant l'ensemble statistique des configurations finales possibles. Si $\mathcal P(q)$ est la probabilité pour que la carte connue finisse à la position $q$, l'entropie est donnée par la formule de Shannon:

$$ S_1(p) = - \sum_q \mathcal P(q) \ln \mathcal P(q) $$

Pour un paquet de 52 cartes, la valeur maximale de $S_1$ est atteinte lorsque $\mathcal P(q)=1/52$ et vaut:

$$ S_1^{max}(p) = - 52 \cdot \frac{1}{52} \cdot \ln\frac{1}{52} = \ln 52 \approx 3.951 $$

### Entropie à $n$ positions: $S_n$

Similairement, nous pouvons définir l'entropie associée au mélange d'un paquet de cartes de $n$ cartes aux positions connues $(p_1,\cdots,p_n)$. Si $\mathcal P(q_1,\cdots,q_n)$ désigne la probabilité d'obtenir la configuration finale $(q_1,\cdots,q_n)$, nous avons l'entropie:

$$ S_n(p_1,\cdots,p_n) = - \sum_{q_1,\cdots,q_n} \mathcal P(q_1,\cdots,q_n) \ln\mathcal P(q_1,\cdots,q_n)$$

Pour un paquet de 52 cartes, la valeur maximale de $S_2$ est atteinte lorsque $\mathcal P(q_1,q_2) = 1/(52\cdot 51)$ et vaut:

$$ S^{max}_2(p_1,p_2) = -52\cdot 51 \cdot \frac 1 {52 \cdot 51} \cdot \ln \frac 1 {52 \cdot 51} = \ln (52 \cdot 51) \approx 7.883 $$

## Implémentations de mélanges

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
import time,pandas

### Mélange "haut bas"

In [ ]:
def melange_1(paquet,n,tmin,tmax):
    
    """
    Fonction de mélange d'un paquet de cartes à la manière "haut-bas"

    Paramètres
    ----------
    1: Le paquet à mélanger, sous forme d'un vecteur numpy.array d'entiers de 1 à #cartes
    2: Nombre de mélanges à effectuer, un entier
    3: Taille minimale des petits paquets de cartes à faire passer du paquet du dessus (ancien) 
       au paquet du dessous (nouveau). Ce nombre sera tiré uniformément entre cette taille minimale et la 
    4: taille maximale utilisée.

    Renvoie:
    --------
    Le paquet mélangé, toujours sous forme d'un vecteur numpy.array d'entiers de 1 à #cartes
    """

    N = len(paquet)
    paquet_nouv = numpy.array([])
    paquet_reste = paquet

    for i in range(n):
        
        t = int(numpy.random.uniform(tmin,tmax+1))
        paquet_nouv = paquet_reste[:t]
        paquet_reste = paquet_reste[t:]
        
        while(len(paquet_reste)>tmax):
            
            t = int(numpy.random.uniform(tmin,tmax+1))
            paquet_nouv = numpy.concatenate((paquet_reste[:t],paquet_nouv))
            paquet_reste = paquet_reste[t:]
            
        paquet_nouv = numpy.concatenate((paquet_reste,paquet_nouv))
        paquet_reste = paquet_nouv
        
    return paquet_nouv

In [ ]:
paquet = numpy.arange(1,53)
print(paquet)

print(melange_1(paquet,1,5,15))

### Mélange "alterné"

In [ ]:
tmax = 4
p = 1/7

hist = numpy.zeros(52)

for i in range(10000):
    
    m = int(numpy.random.binomial(tmax,p))+1
    hist[m] += 1
    
pyplot.plot(hist[:10]);

In [ ]:
def melange_2(paquet,n,tmax,p):
    
    """
    Fonction de mélange d'un paquet de cartes à la manière "alternée" (riffle shuffle)

    Paramètres
    ----------
    1: Le paquet à mélanger, sous forme d'un vecteur numpy.array d'entiers de 1 à #cartes
    2: Nombre de mélanges à effectuer, un entier
    3: Taille maximale des petits paquets de cartes qui s'alternent durant le mélange.
       Ce nombre sera tiré selon une binomiale entre 1 et la taille maximale indiquée avec la
    4: Probabilité pour la binomiale tirant la taille des petits paquets
    
    Renvoie:
    --------
    Le paquet mélangé, toujours sous forme d'un vecteur numpy.array d'entiers de 1 à #cartes
    """

    N = len(paquet)
    paquet_ancien = paquet

    for i in range(n):
        
        paquet_nouv = numpy.array([])
        
        m = int(numpy.random.uniform(N/2-N/12,N/2+N/12+1))
        if(numpy.random.uniform(0,1)>0.5):
            paquet_moitie_1 = paquet_ancien[:m]
            paquet_moitie_2 = paquet_ancien[m:]
        else:
            paquet_moitie_2 = paquet_ancien[:m]
            paquet_moitie_1 = paquet_ancien[m:]
        
        while(len(paquet_moitie_1)>tmax and len(paquet_moitie_2)>tmax):
            
            t1 = int(numpy.random.binomial(tmax,p))+1
            t2 = int(numpy.random.binomial(tmax,p))+1
            paquet_nouv = numpy.concatenate((paquet_moitie_2[-t2:],paquet_moitie_1[-t1:],paquet_nouv))
            paquet_moitie_1 = paquet_moitie_1[:-t1]
            paquet_moitie_2 = paquet_moitie_2[:-t2]
            
        paquet_nouv = numpy.concatenate((paquet_moitie_2,paquet_moitie_1,paquet_nouv))
        paquet_ancien = paquet_nouv
                
    return paquet_nouv

In [ ]:
paquet = numpy.arange(1,53)
print(paquet)

print(melange_2(paquet,1,4,1/7))
print(melange_2(paquet,7,4,1/7))

## Implémentation des entropies positionnelles

### Entropie à 1 position

In [ ]:
def entropie_1(N,melange,p,tps):

    """
    Entropie à 1 position
    
    Paramètres:
    -----------
    1: Taille du paquet à mélanger
    2: Procédure de mélange
    3: Position initiale de la carte connue
    4: Nombre de simulations par configuration finale
    
    Renvoie:
    --------
    1: L'entropie à la position donnée
    2: La densité de probabilité sur les configurations finales
    """
    
    M = int(N*tps) # nombre de simulations pour effectuer l'histogramme des configurations observées
    
    paquet_initial = numpy.arange(1,N+1)
    histogramme = numpy.zeros(N)
    
    for i in range(M):
        
        paquet_melange = melange(paquet_initial)
        histogramme[numpy.where(paquet_melange==p)] += 1
        
    probas = histogramme/M
    log_probas = numpy.zeros(len(probas))

    for i in range(len(probas)):
        if probas[i]!=0:
            log_probas[i] = numpy.log(probas[i])

    entropie = - numpy.sum(probas*log_probas)
    
    return (entropie,probas)

### Valeurs typiques de l'entropie à 1 position pour différentes  distributions finales

Afin de pouvoir se donner une idée de la forme de la distribution finale à partir de la valeur de $S_1$, nous calculons cette entropie pour différentes distributions:

In [ ]:
fig,ax = pyplot.subplots(2,2,figsize=(15,10))

def melange(paquet): return melange_1(paquet,10,5,15)
#def melange(paquet): return melange_2(paquet,1,4,1/7)
(entropie,probas) = entropie_1(52,melange,5,300)
ax[0,0].plot(probas);
ax[0,0].set_title("entropie = {:.4f}".format(entropie));

def melange(paquet): return melange_1(paquet,20,5,15)
#def melange(paquet): return melange_2(paquet,2,4,1/7)
(entropie,probas) = entropie_1(52,melange,5,300)
ax[0,1].plot(probas);
ax[0,1].set_title("entropie = {:.4f}".format(entropie));

def melange(paquet): return melange_1(paquet,30,5,15)
#def melange(paquet): return melange_2(paquet,3,4,1/7)
(entropie,probas) = entropie_1(52,melange,5,300)
ax[1,0].plot(probas);
ax[1,0].set_title("entropie = {:.4f}".format(entropie));

def melange(paquet): return melange_1(paquet,50,5,15)
#def melange(paquet): return melange_2(paquet,5,4,1/7)
(entropie,probas) = entropie_1(52,melange,5,1000)
ax[1,1].plot(probas);
ax[1,1].set_title("entropie = {:.4f}".format(entropie));

Nous voyons que le mélange devient de bonne qualité qu'à partir de $S_1 \approx 3.94$

### Entropie à 2 positions

In [ ]:
def entropie_2(N,melange,p1,p2,tps):

    """
    Entropie à 2 positions
    
    Paramètres:
    -----------
    1: Taille du paquet à mélanger
    2: Procédure de mélange
    3: Position initiale de la carte connue #1
    4: Position initiale de la carte connue #2
    5: Nombre de simulations par configuration finale
    
    Renvoie:
    --------
    1: L'entropie au couple de positions donné
    2: La densité de probabilité sur les configurations finales
    """
    
    M = int(N*(N-1)*tps) # nombre de simulations pour effectuer l'histogramme des configurations observées
    
    paquet_initial = numpy.arange(1,N+1)
    histogramme = numpy.zeros((N,N))
    
    for i in range(M):
        
        # indicateur de progression
        #if (i%(N*(N-1)*10)==0):
        #    print(i/(N*(N-1)),"/100")
            
        paquet_melange = melange(paquet_initial)
        histogramme[numpy.where(paquet_melange==p1),numpy.where(paquet_melange==p2)] += 1
        
    probas = histogramme/M
    log_probas = numpy.zeros((len(probas[:,0]),len(probas[0,:])))

    for i in range(len(probas[:,0])):
        for j in range(len(probas[0,:])):
            
            if probas[i][j]!=0:
                log_probas[i][j] = numpy.log(probas[i][j])

    entropie = - numpy.sum(probas*log_probas)
        
    return (entropie,probas)

### Valeurs typiques de l'entropie à 2 positions pour différentes  distributions finales

In [ ]:
fig,ax = pyplot.subplots(2,2,figsize=(15,10))

#def melange(paquet): return melange_1(paquet,10,5,15)
def melange(paquet): return melange_2(paquet,3,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,10)
ax[0,0].set_title("entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[0,0].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[0,0])

#def melange(paquet): return melange_1(paquet,20,5,15)
def melange(paquet): return melange_2(paquet,5,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,50)
ax[0,1].set_title("entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[0,1].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[0,1])

#def melange(paquet): return melange_1(paquet,30,5,15)
def melange(paquet): return melange_2(paquet,7,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,50)
ax[1,0].set_title("entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[1,0].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[1,0])

#def melange(paquet): return melange_1(paquet,100,5,15)
def melange(paquet): return melange_2(paquet,9,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,100)
ax[1,1].set_title("entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[1,1].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[1,1])

pyplot.show()

Le mélange est de bonne qualité qu'à partir de $S_2 \approx 7.86$.

## Sources d'erreur dans les mesures d'entropie

### Nombre insuffisant de simulations

Si le nombre de simulations par configuration finale à échantilloner n'est pas assez grand, peu de données seront récoltée pour déterminer la densité de probabilité en un site. En conséquence, une grande variance sera observée sur les probabilités en chaque site et l'entropie sera sous-estimée. 

Un exemple est présenté ci-dessous, où l'on calcule l'entropie à 1 position pour un mélange réalisé par la fonction \texttt{numpy.shuffle}. Le mélange est censé être parfait, donnant des configurations finales équiprobables et l'entropie doit donc prendre sa valeur maximale de $S_1 \approx 3.951$. Pourtant, suivant le nombre de simulations par site (tps) que nous effectuons, nous observons des valeurs plus petites de l'entropie à cause du nombre limité de données. 

In [ ]:
fig,ax = pyplot.subplots(2,2,figsize=(15,10))

def melange(paquet): 
    numpy.random.shuffle(paquet)
    return paquet

print("Valeur maximale de S1 = ",numpy.log(52))

(entropie,probas) = entropie_1(52,melange,5,10)
ax[0,0].plot(probas);
ax[0,0].set_title("tps = 10, entropie = {:.6f}".format(entropie));

(entropie,probas) = entropie_1(52,melange,5,100)
ax[0,1].plot(probas);
ax[0,1].set_title("tps = 100, entropie = {:.6f}".format(entropie));

(entropie,probas) = entropie_1(52,melange,5,1000)
ax[1,0].plot(probas);
ax[1,0].set_title("tps = 1000, entropie = {:.6f}".format(entropie));

(entropie,probas) = entropie_1(52,melange,5,10000)
ax[1,1].plot(probas);
ax[1,1].set_title("tps = 10000, entropie = {:.6f}".format(entropie));

Ci-dessous un example plus concret avec un vrai mélange.

In [ ]:
fig,ax = pyplot.subplots(2,2,figsize=(15,10))

def melange(paquet): return melange_1(paquet,20,5,15)
(entropie,probas) = entropie_1(52,melange,5,10)
ax[0,0].plot(probas);
ax[0,0].set_title("tps = 10, entropie = {:.4f}".format(entropie));

def melange(paquet): return melange_1(paquet,20,5,15)
(entropie,probas) = entropie_1(52,melange,5,100)
ax[0,1].plot(probas);
ax[0,1].set_title("tps = 100, entropie = {:.4f}".format(entropie));

def melange(paquet): return melange_1(paquet,20,5,15)
(entropie,probas) = entropie_1(52,melange,5,1000)
ax[1,0].plot(probas);
ax[1,0].set_title("tps = 1000, entropie = {:.4f}".format(entropie));

def melange(paquet): return melange_1(paquet,20,5,15)
(entropie,probas) = entropie_1(52,melange,5,10000)
ax[1,1].plot(probas);
ax[1,1].set_title("tps = 10000, entropie = {:.4f}".format(entropie));

Voyons ce qui se passe dans le cas de l'entropie à 2 positions.

In [ ]:
fig,ax = pyplot.subplots(2,2,figsize=(15,10))

def melange(paquet): 
    numpy.random.shuffle(paquet)
    return paquet

print("Valeur maximale de S2 = ",numpy.log(52*51))

(entropie,probas) = entropie_2(52,melange,5,6,1)
ax[0,0].set_title("tps = 1, entropie = {:.4f}".format(entropie));

for i in range(52):
    probas[i,i] = numpy.nan
    
x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[0,0].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[0,0])

(entropie,probas) = entropie_2(52,melange,5,6,10)
ax[0,1].set_title("tps = 10, entropie = {:.4f}".format(entropie));

for i in range(52):
    probas[i,i] = numpy.nan
    
x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[0,1].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[0,1])

(entropie,probas) = entropie_2(52,melange,5,6,100)
ax[1,0].set_title("tps = 100, entropie = {:.4f}".format(entropie));

for i in range(52):
    probas[i,i] = numpy.nan
    
x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[1,0].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[1,0])

(entropie,probas) = entropie_2(52,melange,5,6,1000)
ax[1,1].set_title("tps = 1000, entropie = {:.4f}".format(entropie));

for i in range(52):
    probas[i,i] = numpy.nan

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[1,1].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[1,1])

pyplot.show()

Un cas plus concret:

In [ ]:
fig,ax = pyplot.subplots(2,2,figsize=(15,10))

#def melange(paquet): return melange_1(paquet,10,5,15)
def melange(paquet): return melange_2(paquet,5,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,1)
ax[0,0].set_title("tps = 1, entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[0,0].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[0,0])

#def melange(paquet): return melange_1(paquet,20,5,15)
def melange(paquet): return melange_2(paquet,5,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,10)
ax[0,1].set_title("tps = 10, entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[0,1].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[0,1])

#def melange(paquet): return melange_1(paquet,30,5,15)
def melange(paquet): return melange_2(paquet,5,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,50)
ax[1,0].set_title("tps = 50, entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[1,0].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[1,0])

#def melange(paquet): return melange_1(paquet,100,5,15)
def melange(paquet): return melange_2(paquet,5,4,1/7)
(entropie,probas) = entropie_2(52,melange,5,6,100)
ax[1,1].set_title("tps = 100, entropie = {:.4f}".format(entropie));

x = numpy.arange(1,53)
y = numpy.arange(1,53)
nrows, ncols = 52, 52
grid = probas.reshape((nrows, ncols))
p = ax[1,1].imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', cmap=cm.viridis)
fig.colorbar(p, shrink=0.8,ax=ax[1,1])

pyplot.show()

Ces examples sont utiles pour choisir le bon nombre de simulations par site pour que l'erreur causée par les fluctuations ne soit pas du même ordre que l'écart entre la valeur de l'entropie mesurée et la valeur d'entropie maximale.

## Stockage dans des fichiers

Fonctions nous permettant de stocker et lire les vecteurs et matrices des résultats dans un fichier externe.

In [ ]:
def storevec(vec, filename):
    
    M = len(vec)
    
    data = list(zip(vec))
    df = pandas.DataFrame(data = data,columns = ["vec"])
    df.to_csv(filename,index=False,header=True)
    
    
def readvec(filename):
    
    data = pandas.read_csv(filename)
    [vec] = numpy.transpose(data.as_matrix(["vec"]))
    
    return vec


def mat2vec(mat):
    
    N = len(mat[0,:])
    M = len(mat[:,0])
    
    vec = numpy.zeros(N*M)
    for i in range(N):
        vec[i*M:i*M+M] = mat[i,:]
        
    return vec


def vec2mat(vec,M):
    
    N = int(len(vec)/M)
    
    mat = numpy.zeros((N,M))
    for i in range(N):
        mat[i,:] = vec[i*M:i*M+M]
        
    return mat

def storemat(mat, filename):
    
    M = len(mat[0,:])
    vec = mat2vec(mat)
    
    data = list(zip(vec))
    df = pandas.DataFrame(data = data,columns = ["vec"])
    df.to_csv(filename,index=False,header=True)
    
    
def readmat(filename,M):
    
    data = pandas.read_csv(filename)
    [vec] = numpy.transpose(data.as_matrix(["vec"]))
    
    mat = vec2mat(vec,M)
    return mat